In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('adult.data.csv')
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [6]:
# How many of each race are represented in this dataset?
# This should be a Pandas series with race names as the index labels.
race_count = data.groupby('race')['race'].count()
race_count

race
Amer-Indian-Eskimo      311
Asian-Pac-Islander     1039
Black                  3124
Other                   271
White                 27816
Name: race, dtype: int64

In [4]:
# What is the average age of men?
average_age_men = data[data['sex'] == 'Male']['age'].mean().round(decimals=2)
average_age_men

39.43

In [5]:
# What is the percentage of people who have a Bachelor's degree?
percentage_bachelors = pd.DataFrame(data={'grade':data['education'].unique()})
percentage_bachelors['quantity'] = data.groupby('education', as_index=False)['education'].count()
total = percentage_bachelors['quantity'].sum()
percentage_bachelors['percentage %'] = ((percentage_bachelors['quantity'] / total) * 100).round(decimals=2)
percentage_bachelors

,grade,quantity,percentage %
0,Bachelors,933,2.87
1,HS-grad,1175,3.61
2,11th,433,1.33
3,Masters,168,0.52
4,9th,333,1.02
5,Some-college,646,1.98
6,Assoc-acdm,514,1.58
7,Assoc-voc,1067,3.28
8,7th-8th,1382,4.24
9,Doctorate,5355,16.45


In [6]:
# What percentage of people with advanced education
#(`Bachelors`, `Masters`, or `Doctorate`) make more than 50K?
edu_levels = ['Bachelors', 'Masters', 'Doctorate']
total = data['education'].isin(edu_levels).count()
advanced = data.loc[(data['education'].isin(edu_levels)) & (data['salary'] == '>50K')][['education', 'salary']]
advanced_values = advanced['education'].value_counts().values
advanced_edu = pd.DataFrame({'education': advanced['education'].unique(), 'total': advanced_values, 'percentage %': ((advanced_values / total) * 100).round(decimals=2)})
advanced_edu

,education,total,percentage %
0,Masters,2221,6.82
1,Bachelors,959,2.95
2,Doctorate,306,0.94


In [7]:
# What percentage of people without advanced education make more than 50K?

# DataFrame of grades but `Bachelors`, `Masters`, or `Doctorate`
low_edu_more = pd.DataFrame(data.loc[(data['education'].values) & (data['salary'] == '>50K')][['education', 'salary']].value_counts()).reset_index()
low_edu_more = low_edu_more.drop(low_edu_more[low_edu_more['education'].isin(edu_levels)].index).reset_index(drop=True)

# Total of people with studies
total = data['education'].count()
total_low_edu = low_edu_more['count'].sum()

# Percentage of people with low education by every grade
low_edu_more['percentage %'] = ((low_edu_more['count'] / total) * 100).round(decimals=2)

# Percentage of people with low education by total and low grades
low_grades_perc = pd.DataFrame({'total grades': total, '% low grades > $50k': ((total_low_edu / total) * 100).round(decimals=2)}, index=['count'])
low_grades_perc

,total grades,% low grades > $50k
count,32561,13.37


In [43]:
# percentage with salary >50K

# Total of people with studies
total = data['education'].count()

# DataFrame of the total count of education grades
higher_education_rich = pd.DataFrame(data[data['salary'] == '>50K']['education']).value_counts().reset_index()
higher_education_rich.rename(columns={'count':'salary >50K'}, inplace = True)

# Percentages
higher_education_rich['% salary >50K'] = ((higher_education_rich['salary >50K'] / total) * 100).round(decimals=2)
higher_education_rich

lower_education_rich = higher_education_rich[higher_education_rich['% salary >50K'] == higher_education_rich['% salary >50K'].min()]
lower_education_rich

,education,salary >50K,% salary >50K
14,1st-4th,6,0.02


In [56]:
# What is the minimum number of hours a person works per week (hours-per-week feature)?
min_work_hours = data[['hours-per-week']].min()
min_work_hours

hours-per-week    1
dtype: int64

In [97]:
# What percentage of the people who work the minimum number of hours per week have a salary of >50K?

# Total people that work the minimum number of hours per week
total_people = data[data['hours-per-week'] == 40]['hours-per-week'] .count()

# Total people that work the minimum number of hours per week and earns >50k
total_max_salary = data[data['salary'] == '>50K']['salary'].count()

rich_percentage = ((total_max_salary / total_people) * 100).round(decimals=2)
rich_percentage


51.53

In [242]:
# What country has the highest percentage of people that earn >50K?
highest_earning_country_df = pd.DataFrame(data[data['salary'] == '>50K']['native-country'].value_counts()).reset_index()
highest_earning_country_df['% salary >50K'] = ((highest_earning_country_df['count'] / highest_earning_country_df['count'].sum()) * 100).round(decimals=2)
highest_earning_country = highest_earning_country_df[highest_earning_country_df['count'] == highest_earning_country_df['count'].max()]
highest_earning_country_percentage = highest_earning_country['% salary >50K']

highest_earning_country
# highest_earning_country_percentage

,native-country,count,% salary >50K
0,United-States,7171,91.46


In [245]:
# Identify the most popular occupation for those who earn >50K in India.
occ_india = data[(data['native-country'] == 'India') & (data['salary'] == '>50K')]['occupation'].value_counts().reset_index()
top_IN_occupation = occ_india[occ_india['count'] == occ_india['count'].max()]
top_IN_occupation

,occupation,count
0,Prof-specialty,25
